In [ ]:
GTA_TRAIN_CRASH_IMAGES = [
    "1xTqFxguYxvF8zf7_e_clAq2GTq1Wi3vC",
    "1K_wEYFvyqMI_Dq_Au8d97Fb23RgCzS40",
    "1JjuJ-h670FWYsaZ5V7XMGPNg8QNertRU", 
]

YT_TRAIN_CRASH_IMAGES = [
    "1mC1nlkLa6ffU09ChbQLK7d7sN_B-rMxP"
]

GTA_TRAIN_SAFE_IMAGES = [
    "1fSEqEvhDm-vKm4ZPSddn08w6NuV3Zm4l",
    "1Q8xQMWrdbzSTjCa8Cr68Cnx1gN5DxTfP",
]

YT_TRAIN_SAFE_IMAGES = [
    "1Ofpe5ZKlf3pWix8Ho3Zm-7ZpxEIH_8qL"
]

TEST_CRASH_IMAGES = [
    "1xGn2O-N6_ONAwWnbjQKSpcTloU6QLapR",
]

TEST_SAFE_IMAGES = [
    "15VMXowVcbZVVm7aOw2wsgCuzqmUMZanX"
]

In [ ]:
GTA_TRAIN_CRASH_LABELS = [
    "1LKHVBPeadPzbMZjsXyALD0ERMyayq1vv",
    "1CJmC21G4UOM1B2WnMax_aIkvk9X-Ho-R",
    "1Ikgkpl4EbZga2IrymInWlzyUmuWNK9eB",
]

YT_TRAIN_CRASH_LABELS = [
    "1lkeswrHasRF1tH6selq5kp-JTbCvfkjS"
]

GTA_TRAIN_SAFE_LABELS = [
    "13G_MUZ00dw12YhJE-oxCI0KeH8UGzoWR",
    "1zNs9YUDzXzYLuUunTCmQQPCHm51qjHHv",
]

YT_TRAIN_SAFE_LABELS = [
    "1q-VaEHBgSCb_ugob7TPVZrqQX_9CtdUc"
]

TEST_CRASH_LABELS = [
    "11fdzKTSensNlwQpau0Gi8Ikraf0yhYcQ",
]

TEST_SAFE_LABELS = [
    "1lLEY3Gj6sWJCDcGLi0RYomeWCtTHXQIt"
]

In [ ]:
import subprocess 
import gdown
import pandas as pd
import numpy as np
import shutil, os

columns = ['id','vid', 'fid', 'filename', 
           'acceleration','speed', 
           'angularVelocity_x', 'angularVelocity_y', 'angularVelocity_z', 
           'forwardV_x', 'forwardV_y', 'forwardV_z', 
           'objectSize_x', 'objectSize_y', 'objectSize_z', 
           'position_x', 'position_y', 'position_z',
           'vehicleCount','vehicleInfo', "target"]
offset = 0

for name in ["GTA", "YT"][]:
    datasets = []
    dst_folder = f"/kaggle/working/{name}/images"
    os.makedirs(dst_folder, exist_ok = True)
    for label in ['CRASH', 'SAFE']:
        print(f"{name = } {label = }")
        src_folder = f"/kaggle/working/{name}_{label}"
        os.makedirs(os.path.join(src_folder, "images"), exist_ok = True)
        os.makedirs(os.path.join(src_folder, "labels"), exist_ok = True)
        for images, labels in zip(
            eval(f"{name}_TRAIN_{label}_IMAGES"), 
            eval(f"{name}_TRAIN_{label}_LABELS")
        ):
            print(f"{images = } {labels = }")
            gdown.download(id = images, output = os.path.join(src_folder, "images.tar.gz"), resume = True)
            subprocess.call(f"tar -xf {os.path.join(src_folder, "images.tar.gz")} --strip-components 1 -C".split() + [os.path.join(src_folder, "images")])
            subprocess.call(f"rm -f".split() + [os.path.join(src_folder, "images.tar.gz")])
            
            gdown.download(id = labels, output = os.path.join(src_folder, "labels.tar.gz"), resume = True)
            subprocess.call(f"tar -xf {os.path.join(src_folder, "labels.tar.gz")} --strip-components {2 if name == "GTA" else 1} -C".split() + [os.path.join(src_folder, "labels")])
            subprocess.call(f"rm -f".split() + [os.path.join(src_folder, "labels.tar.gz")])
    
        dataset, offset = label2csv(src_folder, offset)
        
        if label == "CRASH":
            dataset['target'] = 1
        else:
            dataset['target'] = 0

        shutil.rmtree(os.path.join(src_folder, "labels"))
        for src_file, dst_file in zip(dataset['id'], dataset['filename']):
            shutil.move(
                os.path.join(src_folder, "images", src_file), 
                os.path.join(dst_folder, dst_file)
            )
        # shutil.rmtree(src_folder)
        datasets.append(dataset)

    data = pd.concat(datasets, axis = 0).reset_index(drop = True)
    if all([col in data.columns for col in columns]):
        data = data[columns]
    
    data.to_csv(
        os.path.join(
            os.path.dirname(dst_folder), 
            "train.csv"
        ), 
        index = False
    )
    
    offset = 0
    datasets = []

In [ ]:
offset = 0
name = "test"

datasets = []
dst_folder = f"/kaggle/working/{name}/images"
os.makedirs(dst_folder, exist_ok = True)
for label in ['CRASH', 'SAFE']:
    src_folder = f"/kaggle/working/{name}_{label}"
    os.makedirs(os.path.join(src_folder, "images"), exist_ok = True)
    os.makedirs(os.path.join(src_folder, "labels"), exist_ok = True)
    for images, labels in zip(
        eval(f"TEST_{label}_IMAGES"), 
        eval(f"TEST_{label}_LABELS")
    ):
        gdown.download(id = images, output = os.path.join(src_folder, "images.tar.gz"), resume = True)
        subprocess.call(f"tar -xf {os.path.join(src_folder, "images.tar.gz")} --strip-components 1 -C".split() + [os.path.join(src_folder, "images")])
        subprocess.call(f"rm -f".split() + [os.path.join(src_folder, "images.tar.gz")])
        
        gdown.download(id = labels, output = os.path.join(src_folder, "labels.tar.gz"), resume = True)
        subprocess.call(f"tar -xf {os.path.join(src_folder, "labels.tar.gz")} --strip-components 1 -C".split() + [os.path.join(src_folder, "labels")])
        subprocess.call(f"rm -f".split() + [os.path.join(src_folder, "labels.tar.gz")])

    dataset, offset = label2csv(src_folder, offset)
    
    if label == "CRASH":
        dataset['target'] = 1
    else:
        dataset['target'] = 0

    shutil.rmtree(os.path.join(src_folder, "labels"))
    for src_file, dst_file in zip(dataset['id'], dataset['filename']):
        shutil.move(
            os.path.join(src_folder, "images", src_file), 
            os.path.join(dst_folder, dst_file)
        )
    src_folder
    datasets.append(dataset)

data = pd.concat(datasets, axis = 0).reset_index(drop = True)

data.to_csv(os.path.join(dst_folder, "test.csv"), index = False)